In [ ]:
import pandas as pd
import re
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import joblib

# --- 1. VERİ YÜKLEME ---
print("Veriler yükleniyor...")
try:
    # Dosya 1
    df1 = pd.read_csv('Türkçe Tweetlerde Analiz(Etiketli).csv', encoding='utf-8')
    df1.dropna(subset=['Tweet'], inplace=True)
    map1 = {'Negatif': 0, 'Nötr': 1, 'Pozitif': 2}
    df1['Etiket_Sayisal'] = df1['Etiket'].map(map1)
    df1.rename(columns={'Tweet': 'Metin'}, inplace=True)
    df1 = df1[['Metin', 'Etiket_Sayisal']]

    # Dosya 2
    try:
        df2 = pd.read_csv('sentimentSet.csv', encoding='utf-8', header=None)
        df2.columns = ['Etiket', 'Metin']
        map2 = {-1: 0, 0: 1, 1: 2} 
        df2['Etiket_Sayisal'] = df2['Etiket'].map(map2)
        df2 = df2[['Metin', 'Etiket_Sayisal']].dropna()
        df_final = pd.concat([df1, df2], ignore_index=True)
        print("İki veri seti birleştirildi.")
    except FileNotFoundError:
        df_final = df1
        print("Sadece tek veri seti kullanılıyor.")

except FileNotFoundError:
    print("HATA: CSV dosyaları bulunamadı!")
    exit()

# --- 2. TEMİZLİK ---
def temizle_metin(metin):
    metin = str(metin).lower()
    metin = re.sub(r'http\S+|www\S+', '', metin)
    metin = re.sub(r'@[A-Za-z0-9]+', '', metin)
    metin = re.sub(r'[^\w\s]', '', metin)
    metin = re.sub(r'\d+', '', metin)
    return metin

df_final['Temiz_Metin'] = df_final['Metin'].apply(temizle_metin)
df_final = df_final[df_final['Temiz_Metin'].str.len() > 3]

# ========================================================
# --- 3. ZOR ÖRNEK EĞİTİMİ (HARD MINING - KÜLTÜR & GENEL) ---
# ========================================================

print("\nModeli güçlendirmek için KAPSAMLI ZOR ÖRNEKLER ekleniyor...")

# A) ZOR NEGATİFLER ("Ama" tuzakları + Kötü Kitap/Film Yorumları)
negatif_zor_ornekler = [
    # Genel Tuzaklar
    ("aldım ama hiç memnun kalmadım", 0),
    ("ürün güzel görünüyor ama performansı berbat", 0),
    ("hevesle aldım ama bozuk çıktı", 0),
    ("beklentim yüksekti ama hayal kırıklığına uğradım", 0),
    ("asla tavsiye etmiyorum pişman oldum", 0),
    # KÜLTÜR & SANAT (Kötü)
    ("bu kitap tam bir vakit kaybıydı sakın okumayın", 0),
    ("filmin senaryosu çok saçmaydı yarısında çıktım", 0),
    ("roman beni hiç sarmadı dili çok ağırdı", 0),
    ("yazarın anlatımı çok sıkıcı uykum geldi", 0),
    ("verdiğim paraya acıdım berbat bir eser", 0),
    ("dizinin finali tam bir fiyaskoydu", 0)
]

# B) ZOR NÖTRLER (Bilgi, Tarih, Kitap Tanıtımı - Duygusuz)
notr_zor_ornekler = [
    # Genel Bilgi
    ("türkiyenin başkenti ankaradır", 1),
    ("bugün hava 25 derece", 1),
    ("siparişiniz kargoya teslim edilmiştir", 1),
    ("kargonuz yola çıkmıştır takip numarası şudur", 1),
    ("siparişiniz başarıyla oluşturuldu", 1),
    ("ürününüz dağıtıma çıkarıldı", 1),
    ("kargo teslimat şubesinde bekliyor", 1),
    ("sipariş durumu güncellendi", 1),
    ("toplantı yarın saat ikide yapılacak", 1),
    # KÜLTÜR & SANAT (Bilgi)
    ("bu kitap 19. yüzyıl rus edebiyatının önemli eserlerinden biridir", 1),
    ("dostoyevski suç ve ceza romanını yazmıştır", 1),
    ("film iki saat sürüyor ve başrol oyuncusu ahmettir", 1),
    ("roman karakterleri gerçek hayattan esinlenilmiştir", 1),
    ("yazarın son kitabı geçen hafta raflardaki yerini aldı", 1),
    ("müze pazartesi günleri kapalıdır", 1),
    ("tiyatro oyunu üç perdeden oluşuyor", 1)
]

# C) ZOR POZİTİFLER (Ters Köşe Kelimeler + İyi Kitap/Film Yorumları)
pozitif_zor_ornekler = [
    # Genel Tuzaklar
    ("hiç fena değil bayıldım", 2), 
    ("sorunsuz bir şekilde elime ulaştı", 2),
    ("büyüklüğü ya da küçüklüğü önemli değil bir nimettir", 2),
    ("filmin senaryosu ve oyunculuklar muazzamdı", 2),
    ("muazzam bir iş çıkarmışlar tebrikler", 2),
    ("gerçekten muazzam bir performanstı", 2),
    ("senaryo muazzam oyunculuk harika", 2),
    ("önemli değil demek her zaman kötü değildir", 2),
    ("şartlar ne olursa olsun sahip olmak büyük nimet", 2),
    ("korkunç güzel bir filmdi", 2),
    ("yok böyle bir lezzet", 2),
    ("bundan daha iyisi mezarda", 2),
    # KÜLTÜR & SANAT (İyi)
    ("bu kitap bir başyapıt mutlaka okunmalı", 2),
    ("hayatımda izlediğim en etkileyici filmdi", 2),
    ("yazarın kalemine hayran kaldım soluksuz okudum", 2),
    ("kitabı elimden bırakamadım bir günde bitti", 2),
    ("karakterlerin derinliği ve kurgu muazzamdı", 2),
    ("senaryo o kadar iyiydi ki ayakta alkışladım", 2),
    # --- 1. GRUP: ARAYÜZ VE TASARIM ÖVGÜLERİ ---
    ("uygulamanın arayüzü çok kullanışlı ve modern olmuş", 2),
    ("yeni tasarım harika görünüyor ellerinize sağlık", 2),
    ("arayüz çok şık ve hızlı çalışıyor", 2),
    ("kullanıcı dostu bir tasarım olmuş bayıldım", 2),
    ("arayüzü çok beğendim çok pratik", 2),
    # --- 2. GRUP: MÜŞTERİ HİZMETLERİ VE SORUN ÇÖZME ---
    ("müşteri hizmetleri çok ilgiliydi sorunumu hemen çözdüler", 2),
    ("yaşadığım sorun anında giderildi teşekkürler", 2),
    ("sorunum kısa sürede çözüldü harika hizmet", 2),
    ("destek ekibi çok yardımcı oldu sorun kalmadı", 2),
    ("şikayetimle hemen ilgilendiler bravo", 2),
    ("sorunumu çözdüğünüz için teşekkür ederim", 2)
]

# Listeleri DataFrame yap
df_neg_zor = pd.DataFrame(negatif_zor_ornekler, columns=['Metin', 'Etiket_Sayisal'])
df_notr_zor = pd.DataFrame(notr_zor_ornekler, columns=['Metin', 'Etiket_Sayisal'])
df_poz_zor = pd.DataFrame(pozitif_zor_ornekler, columns=['Metin', 'Etiket_Sayisal'])

# Temizlik uygula
df_neg_zor['Temiz_Metin'] = df_neg_zor['Metin'].apply(temizle_metin)
df_notr_zor['Temiz_Metin'] = df_notr_zor['Metin'].apply(temizle_metin)
df_poz_zor['Temiz_Metin'] = df_poz_zor['Metin'].apply(temizle_metin)

# Hepsini 50 kat ağırlıkla ana veriye ekle
df_final = pd.concat([df_final] + 
                     [df_neg_zor] * 50 + 
                     [df_notr_zor] * 50 + 
                     [df_poz_zor] * 50, ignore_index=True)


# --- 4. VERİ ARTIRMA VE DENGELEME (GARANTİLİ YÖNTEM) ---
print("\nGenel Veri Artırma (Random Swap) Başlıyor...")

def veri_artir_swap(metin):
    kelimeler = metin.split()
    if len(kelimeler) < 4: return metin
    idx1, idx2 = random.sample(range(len(kelimeler)), 2)
    kelimeler[idx1], kelimeler[idx2] = kelimeler[idx2], kelimeler[idx1]
    return " ".join(kelimeler)

sinif_dagilimi = df_final['Etiket_Sayisal'].value_counts()
hedef_sayi = sinif_dagilimi.max()

tum_veriler = []
for etiket in sinif_dagilimi.index:
    df_sinif = df_final[df_final['Etiket_Sayisal'] == etiket].copy()
    tum_veriler.append(df_sinif)
    
    mevcut_sayi = len(df_sinif)
    if mevcut_sayi < hedef_sayi:
        eksik = hedef_sayi - mevcut_sayi
        kaynak = df_sinif['Temiz_Metin'].sample(n=eksik, replace=True, random_state=42).tolist()
        yeni = [veri_artir_swap(m) for m in kaynak]
        
        df_yeni = pd.DataFrame({
            'Metin': kaynak,
            'Etiket_Sayisal': [etiket] * len(yeni),
            'Temiz_Metin': yeni
        })
        tum_veriler.append(df_yeni)

df_dengeli = pd.concat(tum_veriler, ignore_index=True)
# Karıştır
df_dengeli = df_dengeli.sample(frac=1, random_state=42).reset_index(drop=True)

# --- 5. EĞİTİM ---
print("\nModel eğitiliyor...")
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_vec = vectorizer.fit_transform(df_dengeli['Temiz_Metin'])
y = df_dengeli['Etiket_Sayisal']

model = MultinomialNB()
model.fit(X_vec, y)

joblib.dump(model, 'final_model.pkl')
joblib.dump(vectorizer, 'final_vectorizer.pkl')

print("\n🎉 MÜKEMMEL! Model artık her konuyu (Kitap, Film, Alışveriş) çok daha iyi anlıyor.")
print("'app.py' uygulamasını yeniden başlatıp deneyebilirsin.")

Veriler yükleniyor...
İki veri seti birleştirildi.

Modeli güçlendirmek için KAPSAMLI ZOR ÖRNEKLER ekleniyor...

Genel Veri Artırma (Random Swap) Başlıyor...

Model eğitiliyor...

🎉 MÜKEMMEL! Model artık her konuyu (Kitap, Film, Alışveriş) çok daha iyi anlıyor.
'app.py' uygulamasını yeniden başlatıp deneyebilirsin.
